# Vectorized version of the slide 3.4 model

 - It is using the I inelastic selector to iterate
   during the return mapping over all the material
   points simultaneously
 - The gradients of potential functions deliver
   matrices with zero elements - `lambdify` delivers
   inhomogeneous arrays then. Quick fix was using the
   ZERO and ONE variables to include dummy arrays.
   But this produces many zeros for all material points.
   Probably, this indicates the limit of the concept.
   However, for prototyping this is still fine.
 - Would it be possible to use `theano`? It has operators
   for gradients and can run on Tensors. Check it here:
   [theano](https://theano-pymc.readthedocs.io/en/latest/tutorial)


In [ ]:
%matplotlib widget
from bmcs_matmod.slide.vslide_34 import Slide34
import bmcs_matmod.slide.vslide_34 as slide_34
import numpy as np
import matplotlib.pylab as plt

In [ ]:
material_params = dict(
    E_T=1, gamma_T=100, K_T=0, S_T=5, c_T=1, bartau=3, 
    E_N=1, S_N=0.5, c_N = 1, m = 0.01, f_t=3, f_c=20, f_c0=10, eta=0.0 # 0.5
)

# Verify Slide34

In [ ]:
slide = Slide34(**material_params)

To extract the names of the sympy symbols that
can be lambdified or used in a code generation to produce
C code a subclass Cymbol of sp.Symbol is defined
that adds another parameter called codename to be used
in lambdification. Any general material model
contains the property/attribute `state_var_shape`
specifying the key-value pair of the state variable
name and its shape to be allocated.

In [ ]:
slide.state_var_shapes

In [ ]:
discr_shape = (1,)

state = {
    var_name: np.zeros(discr_shape + var_shape, dtype=np.float_)
    for var_name, var_shape in
    slide.state_var_shapes.items()
}

In [ ]:
state

In [ ]:
eps_Ema = np.zeros(discr_shape+(3,), dtype=np.float_)

In [ ]:
w_n1, s_x_n1, s_y_n1 = np.einsum('...a->a...',eps_Ema)

Eps_n = np.array([state[eps_name] for eps_name in slide.Eps_names])
Eps_k = np.copy(Eps_n)
Sig_k = np.array([state[sig_name] for sig_name in slide.Sig_names])

In [ ]:
s_x_n1.shape

In [ ]:
Eps_k.shape
Sig_k.shape

In [ ]:
slide.symb.get_Sig_(w_n1, s_x_n1, s_y_n1, Sig_k, Eps_k)[0].shape

In [ ]:
slide.symb.Sig_

In [ ]:
slide.symb.dSig_dEps_

In [ ]:
ONES = np.ones_like(s_x_n1)
ZEROS = np.zeros_like(s_x_n1)

In [ ]:
dSig_dEps_k = slide.symb.get_dSig_dEps_(w_n1, s_x_n1, s_y_n1, Sig_k, Eps_k, ZEROS, ONES)
dSig_dEps_k.shape

In [ ]:
ix1, ix2 = np.ix_((0,1,2),(0,1,2))

In [ ]:
dSig_dEps_k[ix1,ix2,0]

In [ ]:
H_sig_pi = slide.symb.get_H_sig_pi_(Sig_k)
H_sig_pi.shape

In [ ]:
f_k = slide.symb.get_f_(Eps_k, Sig_k, H_sig_pi)
f_k.shape

In [ ]:
%%capture
df_dSig_k = slide.symb.get_df_dSig_(Eps_k, Sig_k, H_sig_pi, ZEROS, ONES)

In [ ]:
df_dSig_k.shape

In [ ]:
%%capture
ddf_dEps_k = slide.symb.get_ddf_dEps_(Eps_k, Sig_k, H_sig_pi, ZEROS, ONES)

In [ ]:
ddf_dEps_k.shape

In [ ]:
df_dEps_k = np.einsum(
    'ik...,ji...->jk...', df_dSig_k, dSig_dEps_k) + ddf_dEps_k
df_dEps_k.shape

In [ ]:
%%capture
slide.get_f_df(w_n1, s_x_n1, s_y_n1, Sig_k, Eps_k)

In [ ]:
slide.get_Eps_k1(w_n1, s_x_n1, s_y_n1, Eps_n, ONES, Sig_k, Eps_k)

# Compare the single point and multiple point versions

In [ ]:
%matplotlib widget
from bmcs_matmod.slide.vslide_34 import Slide34
import bmcs_matmod.slide.vslide_34 as slide_34
import numpy as np
import matplotlib.pylab as plt

In [ ]:
material_params = dict(
    E_T=1, gamma_T=1, K_T=0, S_T=5, c_T=1, bartau=3, 
    E_N=1, S_N=0.5, c_N = 1, m = 0.01, f_t=3, f_c=20, f_c0=10, eta=0.0 # 0.5
)

In [ ]:
slide = Slide34(**material_params)

In [ ]:
discr_shape = (1,)
state = {
    var_name: np.zeros(discr_shape + var_shape, dtype=np.float_)
    for var_name, var_shape in
    slide.state_var_shapes.items()
}
eps_Ema = np.zeros(discr_shape+(3,), dtype=np.float_) + 1e-9
eps_Ema.shape

In [ ]:
s_max = 4
eps_Ema[0,1] = s_max
slide.k_max = 3
sig_Ema, D_Emab = slide.get_corr_pred(eps_Ema, 1, **state)

Compare the values in the two versions

In [ ]:
sig_Ema, state

In [ ]:
from bmcs_matmod.slide.slide_32 import Slide32
Sig_k_32 = np.zeros((8,))
Eps_k_32 = np.zeros((8,)) + 1e-9
slide32 = Slide32(**material_params)
Eps_k_32, Sig_k_32, k = slide32.get_sig_n1(s_max,0,0,Sig_k_32, Eps_k_32,3)

In [ ]:
Eps_k_32

In [ ]:
Sig_k_32

# Plotting of the elementary monotonic loading

Plot the stress strain curve and evaluate the energy dissipation evolution

In [ ]:
material_params = dict(
    E_T=1000, gamma_T=10, K_T=0, S_T=0.001, c_T=1, bartau=3, 
    E_N=100, S_N=0.01, c_N = 1, m = 0.01, f_t=3, f_c=20, f_c0=10, eta=0.0 # 0.5
)

# Prototyping a method for plotting of the stress strain curve

In [ ]:
slide = Slide34(**material_params)
discr_shape = (1,)
state = {
    var_name: np.zeros(discr_shape + var_shape, dtype=np.float_)
    for var_name, var_shape in
    slide.state_var_shapes.items()
}
n_s = 1000
idx = 0
s_max = .2
s_range = np.linspace(0,s_max,n_s)
tau_range = []
slide.k_max = 20
eps_Ema = np.zeros(discr_shape+(3,), dtype=np.float_) + 1e-9
for i, s in enumerate(s_range):
    eps_Ema[0,idx] = s
    sig_Ema, D_Emab = slide.get_corr_pred(eps_Ema, 1, **state)
    tau_range.append(sig_Ema[0,idx])

In [ ]:
_, ax = plt.subplots(1,1)
ax.plot(s_range, tau_range)

In [ ]:
slide.interact()